In [29]:
import numpy as np
import pandas as pd

In [30]:
file_path = './Model.xlsx'
sheet_name = 'Model'
data = pd.read_excel(file_path, sheet_name=sheet_name)

trimmed_data = "Trimmed_Data"

In [35]:
columns_to_exclude = ['TOTEX', 'URB','emp_status']
columns_to_trim = [col for col in data.columns if col not in columns_to_exclude]


In [36]:
def detect_outliers_iqr(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return ~((column >= lower_bound) & (column <= upper_bound)) 

In [37]:
outlier_mask = pd.DataFrame(False, index=data.index, columns=columns_to_trim)
for col in columns_to_trim:
    outlier_mask[col] = detect_outliers_iqr(data[col])

rows_with_outliers = outlier_mask.any(axis=1)  # Identify rows with any outliers
trimmed_data = data[~rows_with_outliers]

In [38]:
if trimmed_data.empty:
    print("The trimmed_data DataFrame is empty. Nothing to save.")
else:
    print(f"The trimmed_data DataFrame has {len(trimmed_data)} rows and {len(trimmed_data.columns)} columns.")

The trimmed_data DataFrame has 147472 rows and 5 columns.


In [40]:
try:
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        trimmed_data.to_excel(writer, sheet_name=sheet_trimmed, index=False)
    print(f"Trimmed data has been successfully saved to the sheet '{sheet_trimmed}'.")
except Exception as e:
    print(f"Error while writing to Excel: {e}")

Trimmed data has been successfully saved to the sheet 'Trimmed_Data'.
